# Qiskit Coding Tasks: Multi-Qubit Circuits & Noise

This notebook contains three coding exercises based on your Week 5 teaching. These tasks will guide you from building complex gates to simulating noise with the density matrix formalism.

**Setup:**
You will need `qiskit` and `numpy`. Install them if needed:
`pip install qiskit qiskit-aer numpy`

## Setup: Imports for all tasks

In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile, qasm2
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector, DensityMatrix, Operator, Kraus
from numpy import sqrt

# Set up the simulators
# We'll use the statevector simulator for ideal circuits
sv_simulator = AerSimulator(method='statevector')
# We'll use the density_matrix simulator for noisy circuits
dm_simulator = AerSimulator(method='density_matrix')

--- 

## Task 1: The Toffoli Gate & OpenQASM

### 1. Engage (Why?)

*Multiple Means of Engagement (Recruiting Interest)*

To build complex algorithms, we need logic that is more powerful than just a single-controlled CNOT gate. The **Toffoli ($CCX$)** gate is a "doubly-controlled-NOT". It forms the basis of all classical reversible computation and is a key building block in quantum algorithms.

### 2. Explore (Play)

*Multiple Means of Action & Expression (Fluency)*

In Qiskit, adding a Toffoli gate is simple. The command is `qc.ccx(control_qubit_1, control_qubit_2, target_qubit)`. Let's build a simple circuit with it.

In [4]:
# Create a 3-qubit circuit
qc_ccx = QuantumCircuit(3)

# Apply a CCX gate
# q0 and q1 are controls, q2 is the target
qc_ccx.ccx(0, 1, 2)

print("A simple Toffoli circuit:")
print(qc_ccx.draw())

A simple Toffoli circuit:
          
q_0: ──■──
       │  
q_1: ──■──
     ┌─┴─┐
q_2: ┤ X ├
     └───┘


### 3. Explain (The Concept)

*Multiple Means of Representation (Clarify Notation)*

This gate applies an $X$ gate to `q2` *if and only if* `q0` and `q1` are *both* in the $|1\rangle$ state. 
We can also represent this circuit in the **OpenQASM** language. Qiskit can generate this for us automatically.

In [5]:
# Generate the OpenQASM 2.0 string for the circuit
qasm_string = qasm2.dumps(qc_ccx)
print("\nOpenQASM representation:")
print(qasm_string)


OpenQASM representation:
OPENQASM 2.0;
include "qelib1.inc";
qreg q[3];
ccx q[0],q[1],q[2];


### 4. Elaborate (Apply)

*Multiple Means of Action & Expression (Higher-level skills)*

Let's prove the logic. We will prepare the input state $|110\rangle$ and then apply the $CCX$ gate. The final state should be $|111\rangle$.

Note: remember the notation of qiskit is $|q_2q_1q_0\rangle$

**Your Task:** 
1.  Create a 3-qubit `QuantumCircuit`.
2.  Prepare the state $|110\rangle$. (Hint: Apply $X$ gates to `q2` and `q1`).
3.  Add a `barrier` to separate preparation from the main logic.
4.  Apply the `ccx` gate with `q2` and `q1` as controls, `q0` as target.
5.  Use the `Statevector` simulator to get the final state.

In [ ]:
# --- Your Code Here --- 

# 1. & 2. Prepare |110>

# 3. Add barrier

# 4. Apply CCX



# 5. Simulate and get statevector

# .draw('latex') gives a nice visualization. 
# In a text-based environment, just print(final_state)

# The state |111> is the 8th basis vector (index 7)
# --- End Your Code ---

#### Answers

In [6]:
# --- Your Code Here --- 
qc_toffoli_test = QuantumCircuit(3)

# 1. & 2. Prepare |110>
qc_toffoli_test.x(1)
qc_toffoli_test.x(2)

# 3. Add barrier
qc_toffoli_test.barrier()

# 4. Apply CCX
qc_toffoli_test.ccx(1, 2, 0)

print("Test circuit for |110> -> |111>:")
print(qc_toffoli_test.draw())

# 5. Simulate and get statevector
final_state = Statevector(qc_toffoli_test)

print("\nFinal Statevector:")
# .draw('latex') gives a nice visualization. 
# In a text-based environment, just print(final_state)
display(final_state.draw('latex'))
# The state |111> is the 8th basis vector (index 7)
# --- End Your Code ---

Test circuit for |110> -> |111>:
           ░ ┌───┐
q_0: ──────░─┤ X ├
     ┌───┐ ░ └─┬─┘
q_1: ┤ X ├─░───■──
     ├───┤ ░   │  
q_2: ┤ X ├─░───■──
     └───┘ ░      

Final Statevector:


<IPython.core.display.Latex object>

### 5. Evaluate (Challenge)

*Multiple Means of Action & Expression (Monitoring Progress)*

You proved that $CCX|110\rangle = |111\rangle$. 

**Challenge:** How would you modify the code above to prove the *other* case from your notes: $CCX|100\rangle = |100\rangle$? (Hint: You only need to change one line in the "Prepare" step).

In [ ]:
# --- Your Code Here ---

--- 

## Task 2: Pure vs. Mixed States (Density Matrix)


### 1. Engage (Why?)

*Multiple Means of Engagement (Recruiting Interest)*

The `Statevector` we just used is great, but it can't describe a noisy, decohered state. For that, we *must* use the **Density Matrix ($\rho$)**. Let's build the two key examples from your notes: the pure $|+\rangle$ state and the maximally mixed state.

### 2. Explore (Play)

*Multiple Means of Action & Expression (Fluency)*

Qiskit has a `DensityMatrix` object. We can initialize it from a circuit, a label, or a raw matrix. Let's create $\rho_{|0\rangle}$ from the label `'0'`.

In [ ]:
# Create the density matrix for the |0> state
rho_0 = DensityMatrix.from_label('0') # .from_label documentation can referenced here: https://qiskit.org/documentation/stubs/qiskit.quantum_info.DensityMatrix.from_label.html

print("rho_0 = |0><0|")
display(rho_0.draw('latex'))

rho_0 = |0><0|


<IPython.core.display.Latex object>

This matches your notes: $\rho_{|0\rangle} = \begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix}$.

### 3. Explain (The Concept)

*Multiple Means of Representation (Clarify Syntax)*

The $\rho$ for a pure state $|\psi
\rangle$ is $
\rho = |\psi
\rangle\langle\psi|$. For the $|+\rangle$ state, this gives $\rho_{|+\rangle} = \begin{pmatrix} 0.5 & 0.5 \\ 0.5 & 0.5 \end{pmatrix}$. The $0.5$ on the off-diagonals are the **"coherences"**—they represent the superposition.

The maximally mixed state is a *classical* 50/50 mix, $\rho_{mixed} = 0.5 \cdot \rho_{|0\rangle} + 0.5 \cdot \rho_{|1\rangle} = \begin{pmatrix} 0.5 & 0 \\ 0 & 0.5 \end{pmatrix}$. Notice its off-diagonals are **zero**. It has no coherence.

### 4. Elaborate (Apply)

*Multiple Means of Action & Expression (Higher-level skills)*

**Your Task:**
1.  Create the density matrix for the $|+\rangle$ state. (Hint: use `from_label('+')`).
2.  *Manually* create the `DensityMatrix` for the maximally mixed state using a `numpy` array.
3.  Print both.

In [8]:
# --- Your Code Here --- 

# 1. Create rho_plus from its label
rho_plus = DensityMatrix.from_label('+')

# 2. Manually create the numpy array for the mixed state
mixed_array = np.array([
    [0.5, 0],
    [0, 0.5]
])
rho_mixed = DensityMatrix(mixed_array)

# 3. Print them
print("rho_plus = |+><+|")
display(rho_plus.draw('latex'))

print("\nrho_mixed = 0.5*|0><0| + 0.5*|1><1|")
display(rho_mixed.draw('latex'))

# --- End Your Code ---

rho_plus = |+><+|


<IPython.core.display.Latex object>


rho_mixed = 0.5*|0><0| + 0.5*|1><1|


<IPython.core.display.Latex object>

### 5. Evaluate (Challenge)

*Multiple Means of Action & Expression (Monitoring Progress)*

We know that for *any* valid quantum state, $Tr(\rho) = 1$. 

**Challenge:** Use the `.trace()` method to calculate the trace of `rho_plus` and `rho_mixed`. Do they both equal 1?

In [9]:
# --- Your Code Here --- 

# Calculate the trace of the pure state
trace_plus = rho_plus.trace()
print(f"Trace of rho_plus: {trace_plus}")

# Calculate the trace of the mixed state
trace_mixed = rho_mixed.trace()
print(f"Trace of rho_mixed: {trace_mixed}")

# --- End Your Code ---

Trace of rho_plus: (0.9999999999999998+0j)
Trace of rho_mixed: (1+0j)


--- 

## Task 3: Simulating Decoherence (Quantum Channels)


### 1. Engage (Why?)

*Multiple Means of Engagement (Relevance)*

This is the capstone task. We will now *simulate* the decoherence you proved mathematically in your notes. We will build the **Phase Damping Channel** and apply it to the pure $|+\rangle$ state to watch its coherences (off-diagonals) decay.

### 2. Explore (Play)

*Multiple Means of Action & Expression (Fluency)*

A `QuantumChannel` is built from its **Kraus operators ($E_k$)**. From your notes, the Phase Damping Channel with noise probability $p$ has two Kraus operators:

* $E_0 = \sqrt{1-p} \cdot I = \sqrt{1-p} \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}$
* $E_1 = \sqrt{p} \cdot Z = \sqrt{p} \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}$

Let's define these in `numpy` for a *maximum noise* scenario where $p=0.5$.

In [10]:
# Set noise probability
p = 0.5

# Define I and Z matrices
I = np.array([[1, 0], [0, 1]])
Z = np.array([[1, 0], [0, -1]])

# Create the Kraus operators
E0 = sqrt(1-p) * I
E1 = sqrt(p) * Z

# Store them in a list
kraus_ops = [E0, E1]

print("Kraus Operator E0 (sqrt(0.5) * I):")
print(E0)
print("\nKraus Operator E1 (sqrt(0.5) * Z):")
print(E1)

Kraus Operator E0 (sqrt(0.5) * I):
[[0.70710678 0.        ]
 [0.         0.70710678]]

Kraus Operator E1 (sqrt(0.5) * Z):
[[ 0.70710678  0.        ]
 [ 0.         -0.70710678]]


### 3. Explain (The Concept)

*Multiple Means of Representation (Clarify Syntax)*

We now have a list of matrices, `kraus_ops`, that *define* our noise channel. The channel $\mathcal{E}$ will apply the operation $\mathcal{E}(\rho) = E_0 \rho E_0^\dagger + E_1 \rho E_1^\dagger$. 

We can create a `QuantumChannel` object in Qiskit by passing it this list. We can then `evolve` a density matrix using this channel.

### 4. Elaborate (Apply)

*Multiple Means of Action & Expression (Higher-level skills)*

**Your Task:**
1.  Create the initial density matrix for the state $∣+⟩$
2.  Define the Kraus operators (`kraus_ops`) for your desired channel (for example, a phase-damping channel). Then create the channel object.
3.  Evolve the state by applying the channel to `rho_plus`.

In [3]:
# 1. Get the initial pure state ρ₊
rho_plus = DensityMatrix.from_label('+')
print("Initial density matrix (ρ_+):")
print(rho_plus.data)  # The raw numpy array

# 2. Define the channel via Kraus operators:
gamma = 0.5
K0 = np.array([[1, 0],
               [0, np.sqrt(1 - gamma)]])
K1 = np.array([[0, 0],
               [0, np.sqrt(gamma)]])
kraus_ops = [K0, K1]

phase_damping_channel = Kraus(kraus_ops)

# 3. Evolve the state through the channel:
rho_matrix = rho_plus.data   # get raw matrix
rho_final_matrix = sum(K @ rho_matrix @ K.conj().T for K in kraus_ops)
rho_final = DensityMatrix(rho_final_matrix)

print("\nFinal density matrix after channel (ρ_final):")
print(rho_final.data)  # raw numpy array


Initial density matrix (ρ_+):
[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]

Final density matrix after channel (ρ_final):
[[0.5       +0.j 0.35355339+0.j]
 [0.35355339+0.j 0.5       +0.j]]


### 5. Evaluate (The Final Test)

*Multiple Means of Engagement (Mastery)*

This is the moment of truth. Let's print the `rho_final` density matrix. 

**Task:** 
1.  Print `rho_final`. 
2.  **Analyze the result:** Look at the matrix. What happened to the off-diagonal "coherence" elements?

In [4]:
# --- Your Code Here --- 

# 1. Print the final density matrix
print("\nFinal State (after Phase Damping p=0.5):")
display(rho_final.draw('latex'))

# --- End Your Code ---


Final State (after Phase Damping p=0.5):


<IPython.core.display.Latex object>

Now, test the code by changing gamma $\gamma$ with different value. $(0.6, 0.7, 0.8, 0.9999)$